In [2]:
import pandas as pd
import numpy as np

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Utils.DataReader import load_urm, load_icm, load_target
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

In [3]:
URM_all = load_urm()

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.85)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1503 ( 3.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 893 ( 2.1%) Users that have less than 1 test interactions


In [4]:
recommender_class = SLIMElasticNetRecommender

In [5]:
import os

output_folder_path = "Experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 50
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"
cutoff_to_optimize = 10

In [6]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "topK": Integer(1,800),
    "l1_ratio": Real(low = 0.001, high = 0.01, prior = 'log-uniform'),
    "alpha": Real(low = 0.001, high = 0.02, prior = 'log-uniform'),
    #"positive_only": Categorical([True]),
}

In [7]:
earlystopping_keywargs = {"validation_every_n": 15,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_validation,
                          "lower_validations_allowed": 5,
                          "validation_metric": metric_to_optimize,
                          }

In [8]:
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

#create a bayesian optimizer object, we pass the recommender and the evaluator
hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                           evaluator_validation=evaluator_validation)

In [9]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

#provide data needed to create instance of model (one on URM_train, the other on URM_all)
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS ={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_all],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS ={}
)

In [10]:
hyperparameterSearch.search(recommender_input_args = recommender_input_args,
                            recommender_input_args_last_test = recommender_input_args_last_test,
                            hyperparameter_search_space = hyperparameters_range_dictionary,
                            n_cases = n_cases,
                            n_random_starts = n_random_starts,
                            save_model = "last",
                            output_folder_path = output_folder_path, # Where to save the results
                            output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                            metric_to_optimize = metric_to_optimize,
                            cutoff_to_optimize = cutoff_to_optimize,
                            )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 412, 'l1_ratio': 0.004085242368699656, 'alpha': 0.006454058682826422}


SLIMElasticNetRecommender: Processed 6063 (24.7%) in 5.00 min. Items per second: 20.20


SLIMElasticNetRecommender: Processed 12780 (52.1%) in 10.00 min. Items per second: 21.30


SLIMElasticNetRecommender: Processed 19444 (79.3%) in 15.00 min. Items per second: 21.60


SLIMElasticNetRecommender: Processed 24507 (100.0%) in 18.71 min. Items per second: 21.82


...

EvaluatorHoldout: Processed 40126 (100.0%) in 32.98 sec. Users per second: 1217


SearchBayesianSkopt: Config 35 is suboptimal. Config: {'topK': 306, 'l1_ratio': 0.0035658486108087256, 'alpha': 0.004181961739488717} - results: PRECISION: 0.0427204, PRECISION_RECALL_MIN_DEN: 0.0815299, RECALL: 0.0781868, MAP: 0.0195649, MAP_MIN_DEN: 0.0369185, MRR: 0.1415616, NDCG: 0.0730724, F1: 0.0552518, HIT_RATE: 0.3057619, ARHR_ALL_HITS: 0.1657408, NOVELTY: 0.0043029, AVERAGE_POPULARITY: 0.1908674, DIVERSITY_MEAN_INTER_LIST: 0.9479826, DIVERSITY_HERFINDAHL: 0.9947959, COVERAGE_ITEM: 0.1636267, COVERAGE_ITEM_HIT: 0.0501081, ITEMS_IN_GT: 0.9865345, COVERAGE_USER: 0.9638954, COVERAGE_USER_HIT: 0.2947224, USERS_IN_GT: 0.9638954, DIVERSITY_GINI: 0.0197178, SHANNON_ENTROPY: 8.9816003, RATIO_DIVERSITY_HERFINDAHL: 0.9951251, RATIO_DIVERSITY_GINI: 0.0390089, RATIO_SHANNON_ENTROPY: 0.6657819, RATIO_AVERAGE_POPULARITY: 3.7699020, RATIO_NOVELTY: 0.2897087, 



Iteration No: 36 ended. Search finished for the next optimal point.
Time taken: 1310.4695
Function value obtained: -0.0196
Current minimum: -0.0196
Iteration No: 37 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 191, 'l1_ratio': 0.001, 'alpha': 0.003063174578575231}


SLIMElasticNetRecommender: Processed 5190 (21.2%) in 5.00 min. Items per second: 17.29


SLIMElasticNetRecommender: Processed 10861 (44.3%) in 10.00 min. Items per second: 18.10


In [ ]:
from Recommenders.DataIO import DataIO

#explore the results of the search
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

In [ ]:
hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df

In [ ]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

In [ ]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

In [ ]:
recommender = SLIMElasticNetRecommender(URM_train)
recommender.fit(topK=405, l1_ratio=0.0010299956370568744, alpha=0.01)

In [ ]:
recommender.save_model(output_folder_path, file_name = recommender.RECOMMENDER_NAME + "_my_own_save.zip" )

In [ ]:
test_users = pd.read_csv('Dataset/data_target_users_test.csv')

In [ ]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user, cutoff=10))

In [ ]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])

test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('Submissions\Submission_04_RP3Beta_MixedRatings.csv', index=False)